In [1]:
import io
import pandas as pd
import requests
import tweepy
import json
from datetime import datetime
import s3fs
from s3fs import S3FileSystem

In [13]:
import json 
import pandas as pd
import requests
  
subreddit = 'anime'
limit = 10000
timeframe = 'year' #hour, day, week, month, year, all
listing = 'top' # controversial, best, hot, new, random, rising, top
  
def get_reddit(subreddit,listing,limit,timeframe):
    try:
        base_url = f'https://www.reddit.com/r/{subreddit}/{listing}.json?limit={limit}&t={timeframe}'
        request = requests.get(base_url, headers = {'User-agent': 'yourbot'})
    except:
        print('An Error Occured')
    return request.json()
 
def get_post_titles(r):
    '''
    Get a List of post titles
    '''
    posts = []
    for post in r['data']['children']:
        x = post['data']['title']
        posts.append(x)
    return posts
 
def get_results(r):
    '''
    Create a DataFrame Showing Title, URL, Score and Number of Comments.
    '''
    myDict = {}
    for post in r['data']['children']:
        myDict[post['data']['title']] = {'url':post['data']['url'],'score':post['data']['score'],'comments':post['data']['num_comments']}
    df = pd.DataFrame.from_dict(myDict, orient='index')
    return df
 
if __name__ == '__main__':
    r = get_reddit(subreddit,listing,limit,timeframe)
    df = get_results(r)

In [23]:
r = get_reddit(subreddit,listing,limit,timeframe)
df = get_results(r)
df['thread'] = df.index
df = df.reset_index(drop ='True')

In [15]:
df.head(100)
#s3 = S3FileSystem()

,url,score,comments,thread
0,https://onepunchman-anime.net/img/home/visual_...,28179,1282,One Punch Man Season 3 Announced
1,https://i.redd.it/6t7oipcn2he91.jpg,24272,886,'Chainsaw Man' New Key Visual
2,https://www.reddit.com/r/anime/comments/y1cx9q...,23491,4236,Chainsaw Man - Episode 1 discussion
3,https://i.redd.it/uklwaeuoldo91.jpg,21244,475,"""JUJUTSU KAISEN"" Season 2 Teaser Visual"
4,https://i.redd.it/7xsygpc6kzp91.png,20371,399,'Jujutsu Kaisen' Season 2 New Visual
...,...,...,...,...
95,https://i.redd.it/ytuh3a3x61k91.jpg,9720,725,"""Farming Life in Another World"" Teaser Visual"
96,https://www.reddit.com/r/anime/comments/zqtkyg...,9709,2110,Chainsaw Man - Episode 11 discussion
97,https://i.redd.it/uxw7knjhhsn91.jpg,9697,363,"""The Eminence in Shadow"" New Key Visual"
98,https://i.redd.it/r96ax0ot15ma1.png,9678,1197,"Comparing the winners of the r/anime, Crunchyr..."


In [16]:
df.sort_values(by=['comments'], ascending=False).head(20)

,url,score,comments,thread
9,https://www.reddit.com/r/anime/comments/z7y4lc...,16507,4683,Chainsaw Man - Episode 8 discussion
2,https://www.reddit.com/r/anime/comments/y1cx9q...,23491,4236,Chainsaw Man - Episode 1 discussion
23,https://www.reddit.com/r/anime/comments/zeak0l...,14197,3959,Chainsaw Man - Episode 9 discussion
14,https://www.reddit.com/r/anime/comments/12jp7u...,15427,3474,Oshi no Ko - Episode 1 discussion
48,https://www.reddit.com/r/anime/comments/z1xqhu...,11617,2781,Chainsaw Man - Episode 7 discussion
17,https://www.reddit.com/r/anime/comments/y7b2hi...,14864,2532,Chainsaw Man - Episode 2 discussion
33,https://www.reddit.com/r/anime/comments/zwj2h8...,13107,2348,Chainsaw Man - Episode 12 discussion - FINAL
66,https://www.reddit.com/r/anime/comments/yw1gxl...,10751,2245,Chainsaw Man - Episode 6 discussion
32,https://www.reddit.com/r/anime/comments/yd9p6y...,13404,2161,Chainsaw Man - Episode 3 discussion
44,https://www.reddit.com/r/anime/comments/11hjdp...,12076,2144,Shingeki no Kyojin: The Final Season • Attack ...


In [7]:
df.to_csv('s3://reddit-batch-anubhav/animedata.csv', index=False)

In [24]:
df.loc[df['thread'].str.contains("Episode"),'Topic'] = 'Episode'
df.loc[df['thread'].str.contains("visual"),'Topic'] = 'Visual'
df.loc[df['thread'].str.contains("Visual"),'Topic'] = 'Visual'
df.loc[df['thread'].str.contains("Announced"),'Topic'] = 'Announcements'
df.loc[df['thread'].str.contains("announced"),'Topic'] = 'Announcements'

In [22]:
df.count()

url         100
score       100
comments    100
thread      100
dtype: int64

In [25]:
df.loc[df['thread'].str.contains("Suzume"),'Anime'] = 'Suzume'
df.loc[df['thread'].str.contains("Kimetsu"),'Anime'] = 'Demon Slayer'
df.loc[df['thread'].str.contains("Oshi"),'Anime'] = 'Oshi no Ko'
df.loc[df['thread'].str.contains("Chainsaw"),'Anime'] = 'Chainsaw Man'
df.loc[df['thread'].str.contains("Jujutsu"),'Anime'] = 'Jujutsu Kaisen'
df.loc[df['thread'].str.contains("JUJUTSU"),'Anime'] = 'Jujutsu Kaisen'
df.loc[df['thread'].str.contains("One Punch"),'Anime'] = 'One Punch Man'
df.loc[df['thread'].str.contains("Naruto"),'Anime'] = 'Naruto'
df.loc[df['thread'].str.contains("Dragon Ball"),'Anime'] = 'Dragon Ball'
df.loc[df['thread'].str.contains("SPY"),'Anime'] = 'SPY X Family'
df.loc[df['thread'].str.contains("Spy"),'Anime'] = 'SPY X Family'
df.loc[df['thread'].str.contains("Attack"),'Anime'] = 'Attack on Titan'
df.loc[df['thread'].str.contains("ATTACK"),'Anime'] = 'Attack on Titan'